In [1]:
import boto3
import json
import os

In [15]:
import tiktoken

text = """
- The code defines a class called `MyCodebase` which represents a codebase.
- The code initializes the class by setting the directory and establishing a database connection.
- The class has methods for setting and getting the directory, updating files, creating database tables, generating a file tree, and removing old files.
- The code uses the `os` module to interact with the operating system, the `datetime` module to work with dates and times, and the `dotenv` module to load environment variables from a `.env` file.
- The code uses the `tiktoken` library to encode text for the GPT-3.5 Turbo model.
- The code defines constants for the embedding model, encoder, summary model, and readme model.
- The code defines a summary prompt for generating a summary of the code.
- The `MyCodebase` class has a constructor that takes a directory and a database connection as parameters.
- The constructor sets the directory, establishes a database connection, creates tables in the database, updates files and embeddings, and removes old files.
- The class has methods for setting and getting the directory.
- The `update_file` method updates a file in the database with its text, token count, and last modified timestamp.
- The `create_tables` method creates tables in the database for storing files and configuration.
- The `tree` method generates a tree structure of the codebase by fetching file paths from the database and organizing them into a hierarchical structure.
- The `remove_old_files` method removes files from the database that are no longer present in the codebase.
- The `_update_files_and_embeddings` method updates files and embeddings in the codebase by traversing the directory and its subdirectories, filtering out invalid files and directories.
- The `_is_valid_file` method checks if a file is valid based on its name and extension.
- The `_is_valid_directory` method checks if a directory is valid based on its name.
"""

# Count the number of tokens in the text
encoding = tiktoken.get_encoding("cl100k_base")

token_count = len(encoding.encode(text))
print("Token count:", token_count)



Token count: 399


In [4]:
bedrock = boto3.client('bedrock-runtime')

In [7]:
resp = bedrock.invoke_model_with_response_stream(
    modelId='anthropic.claude-v2',
    contentType='application/json',
    accept="*/*",
    body=json.dumps({
        "prompt": "Human: Hello, who are you?\n\nAssistant:",
        "max_tokens_to_sample": 200,
        "temperature": 0.5,
        "top_p": 1,
        "stop_sequences": ["\n\n\n\nHuman:"],
    })
)


In [11]:
i = 0
while True:
    chunk = next(iter(resp['body']))
    print(chunk)
    i +=1
    if i > 100:
        break

{'chunk': {'bytes': b'{"completion":"! I\'m Claude, an AI assistant created by Anthropic.","stop_reason":"stop_sequence"}'}}

In [2]:
plan = command_planner(
'''
The project directory is currently setup as follows:
+--GPT-CodeApp
    +--README.md
    +--frontend
        +--tailwind.config.js
        +--next.config.js
        +--README.md
        +--postcss.config.js
        +--styles
            +--globals.css
        +--components
            +--modal_bar_modals
            +--SearchBar.js
            +--ModelSelector.js
            +--ModalBar.js
            +--ChatInput.js
            +--RightSidebar.js
            +--EditFilesModal.js
            +--ChatBox.js
            +--LeftSidebar.js
        +--store
            +--sidebar
                +--sidebarSlice.js
            +--index.js
            +--messages
                +--messagesSlice.js
        +--pages
            +--index.js
    +--backend
        +--database
            +--UpdateHandler.py
            +--my_codebase.py
        +--agent
            +--openai_function_call.py
            +--memory_manager.py
            +--agent.py
            +--agent_functions.py
        +--.pytest_cache
            +--README.md
        +--tests
            +--.pytest_cache
                +--README.md
            +--test_codebase.py
            +--test_memory_manager.py
        +--main.py

File: frontend/components/ModalBar.js
Content:
import React, { useState, useEffect } from 'react';
import { get_encoding } from "@dqbd/tiktoken";
// import CustomModal from './CustomModal';
import ReactModal from 'react-modal';

ReactModal.setAppElement('#__next');

const encoding = get_encoding("cl100k_base");

const ModalBar = () => {
    const [systemPrompt, setSystemPrompt] = useState("");
    const [systemTokens, setSystemTokens] = useState(0);
    const [isModalOpen, setIsModalOpen] = useState(false);
    const [editablePrompt, setEditablePrompt] = useState("");
    const [functions, setFunctions] = useState([]);
    const [functionTokens, setFunctionTokens] = useState(0);
    const [isFunctionModalOpen, setIsFunctionModalOpen] = useState(false);
    const [messageHistory, setMessageHistory] = useState([]);
    const [isMessageModalOpen, setIsMessageModalOpen] = useState(false);
    const [messageTokens, setMessageTokens] = useState(0);

    const fetchSystemPromptAndOpenModal = (modal = true) => {
        fetch('http://127.0.0.1:8000/system_prompt')
            .then(response => response.json())
            .then(data => {
                setEditablePrompt(data.system_prompt);
                setSystemPrompt(data.system_prompt);
                const tokens = encoding.encode(data.system_prompt);
                setSystemTokens(tokens.length);
                if (modal) {
                    setIsModalOpen(true);
                }
            })
            .catch(console.error);
    };

    const fetchFunctionsAndOpenModal = (modal = true) => {
        fetch('http://127.0.0.1:8000/get_functions')
            .then(response => response.json())
            .then(data => {
                setFunctions(data.functions);
                if (modal) {
                    setIsFunctionModalOpen(true);
                }
                const f_tokens = encoding.encode(JSON.stringify(data.functions[0]));
                setFunctionTokens(f_tokens.length);
            })
            .catch(console.error);
    };

    const fetchMessagesAndOpenModal = (modal = true) => {
        setMessageHistory([]);
        fetch('http://127.0.0.1:8000/get_messages')
            .then(response => response.json())
            .then(data => {
                setMessageHistory(data.messages);
                if (modal) {
                    setIsMessageModalOpen(true);
                }
                let m_tokens = 0;
                for (const message of data.messages) {
                    m_tokens += encoding.encode(message.content).length;
                }
                console.log(m_tokens);
                setMessageTokens(m_tokens);
            })
            .catch(console.error);
    };

    const updateSystemPrompt = (e) => {
        e.preventDefault();
        fetch('http://127.0.0.1:8000/update_system', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ system_prompt: editablePrompt }),
        })
            .then(response => {
                if (!response.ok) throw new Error('Network response was not ok');
                setSystemPrompt(editablePrompt); // Update original systemPrompt
                setIsModalOpen(false); // Close modal
            })
            .catch(console.error);
    };

    useEffect(() => {
        fetchSystemPromptAndOpenModal(false);
        fetchFunctionsAndOpenModal(false);
        fetchMessagesAndOpenModal(false);
    }, []); //
    return (
        <div className='flex flex-row mx-auto pb-3'>
            <div>
                <button onClick={fetchSystemPromptAndOpenModal}>
                    System Prompt:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {systemTokens} tokens
                    </span>
                </button>
                <ReactModal
                    isOpen={isModalOpen}
                    onRequestClose={() => setIsModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="relative bg-white rounded p-4 w-full max-w-screen-lg mx-auto text-gray-900">
                        <h2 className="text-xl">System Prompt</h2>
                        <p className="text-sm text-green-800">Tokens: {systemTokens}</p>
                        <form onSubmit={updateSystemPrompt}>
                            <textarea
                                value={editablePrompt}
                                onChange={(e) => setEditablePrompt(e.target.value)}
                                className="mt-2 w-full h-96 p-2 border border-gray-300 rounded"
                            />
                            <button
                                type="submit"
                                className="mt-4 px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
                            >
                                Update System Prompt
                            </button>
                        </form>
                    </div>
                </ReactModal>
            </div>
            <div className='px-5'>
                < button onClick={fetchFunctionsAndOpenModal} >
                    Functions:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {functionTokens} tokens
                    </span>
                </button >

                <ReactModal
                    isOpen={isFunctionModalOpen}
                    onRequestClose={() => setIsFunctionModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96 w-auto"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="relative bg-white rounded p-4 max-w-screen-lg mx-auto text-gray-900 overflow-scroll">
                        <h2 className="text-xl">Functions</h2>
                        <pre>
                            {functions?.map((f) => (
                                <div key={f?.name}>
                                    <h3 className="text-lg">{f?.name}</h3>
                                    <p>{f?.description}</p>
                                    <hr />
                                </div>
                            ))}
                        </pre>
                    </div>
                </ReactModal>

            </div >
            <div className='px-5'>
                <button onClick={fetchMessagesAndOpenModal}>
                    Message History:
                    <span className="ml-2 inline-block bg-green-500 text-white text-xs px-2 rounded-full uppercase font-semibold tracking-wide">
                        {messageTokens} tokens
                    </span>
                </button>
                <ReactModal
                    isOpen={isMessageModalOpen}
                    onRequestClose={() => setIsMessageModalOpen(false)}
                    shouldCloseOnOverlayClick={true}
                    className="fixed inset-0 flex items-center justify-center m-96"
                    overlayClassName="fixed inset-0 bg-black bg-opacity-50"
                >
                    <div className="bg-gray-700 rounded p-4 max-w-screen-lg max-h-96 overflow-y-scroll mx-auto text-gray-200">
                        <h2 className="text-xl text-white">Messages</h2>
                        {messageHistory.map((m, index) => (
                            <div key={index} className={m.role === 'user' ? 'my-4 p-2 rounded bg-blue-600 text-white' : 'my-4 p-2 rounded bg-green-600 text-white'}>
                                <h3 className="text-lg font-bold">{m.role.toUpperCase()}</h3>
                                <pre className="text-md whitespace-pre-wrap">{m.content}</pre>
                            </div>
                        ))}
                    </div>
                </ReactModal>
            </div>
        </div >
    );
}

export default ModalBar;

INSTRUCTIONS:
In the modal_bar_modals directory (create if necessary), create the following files:
    .../SystemPromptModal.js: This file will contain the code for the system prompt modal.
    .../FunctionsModal.js: This file will contain the code for the functions modal.
    .../MessageHistoryModal.js: This file will contain the code for the message history modal.
    
Move modal code: Extract the code for each modal from ModalBar.js and move it into the corresponding new file. Make sure to maintain the same functionality and state management in each new component.

Refactor ModalBar.js: In ModalBar.js, refactor the code to import and use the new modal components. Replace the inline modal code with component tags, like <SystemPromptModal />, <FunctionsModal />, and <MessageHistoryModal />.

Export new components: Make sure to export each new component from its file using export default ComponentName.

Test the refactored code: After refactoring, thoroughly test the application to ensure that the modals still work as expected. Check that the state is managed correctly and that all user interactions produce the expected results.

Use a new git branch with a random name. Commit and push changes: Once you've verified that everything is working correctly, commit your changes and push them to the repository. Include a descriptive commit message outlining the changes you've made.

Please let me know if you need any further clarification or have any other questions.
'''
)

In [3]:
plan

CommandPlan(command_graph=[Command(id=1, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/SystemPromptModal.js', description='This file will contain the code for the system prompt modal.', reference_files=['frontend/components/ModalBar.js'])), Command(id=2, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/FunctionsModal.js', description='This file will contain the code for the functions modal.', reference_files=['frontend/components/ModalBar.js'])), Command(id=3, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/MessageHistoryModal.js', description='This file will contain the code for the message history modal.', reference_files=['fr

In [4]:
plan.execute()

[Command(id=1, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/SystemPromptModal.js', description='This file will contain the code for the system prompt modal.', reference_files=['frontend/components/ModalBar.js'])), Command(id=2, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/FunctionsModal.js', description='This file will contain the code for the functions modal.', reference_files=['frontend/components/ModalBar.js'])), Command(id=3, command_type=<CommandType.NEW_FILE: 'new_file'>, dependent_commands=[], command_line=None, file_change=None, new_file=NewFile(name='frontend/components/modal_bar_modals/MessageHistoryModal.js', description='This file will contain the code for the message history modal.', reference_files=['frontend/components/ModalBar

{1: CommandResult(command_id=1, output='file created'),
 2: CommandResult(command_id=2, output='file created'),
 3: CommandResult(command_id=3, output='file created'),
 4: CommandResult(command_id=4, output='changes complete'),
 5: CommandResult(command_id=5, output='\r\nThe default interactive shell is now zsh.\r\nTo update your account to use zsh, please run `chsh -s /bin/zsh`.\r\nFor more details, please visit https://support.apple.com/kb/HT208050.\r\nbash-3.2'),
 6: CommandResult(command_id=6, output=' cd /Users/josephblazick/Documents/GPT-CodeApp\r\nbash-3.2'),
 7: CommandResult(command_id=7, output=" npm test\r\ngit checkout -b random_branch_name\r\ngit add .\r\n\x1bnpm\x1b \x1b\x1bERR!\x1b \x1b\x1bcode\x1b ENOENT\r\n\x1b\x1bnpm\x1b \x1b\x1bERR!\x1b \x1b\x1bsyscall\x1b open\r\n\x1b\x1bnpm\x1b \x1b\x1bERR!\x1b \x1b\x1bpath\x1b /Users/josephblazick/Documents/GPT-CodeApp/package.json\r\n\x1b\x1bnpm\x1b \x1b\x1bERR!\x1b \x1b\x1berrno\x1b -2\r\n\x1b\x1bnpm\x1b \x1b\x1bERR!\x1b \x1b\x1

In [9]:
from diff_match_patch import patch_obj, diff_match_patch

DIFF_DELETE = -1
DIFF_INSERT = 1
DIFF_EQUAL = 0

# Initialize a patch object
patch = patch_obj()

# Set the attributes
patch.start1 = 0  # The start of the change in the first text
patch.start2 = 0  # The start of the change in the second text
patch.length1 = 5  # The length of the change in the first text
patch.length2 = 2  # The length of the change in the second text

# Add diffs
# Each diff is a tuple of an operation and the text involved in the operation
# The operation is one of DIFF_INSERT, DIFF_DELETE, DIFF_EQUAL
patch.diffs = [
    (DIFF_DELETE, "Hello"),
    (DIFF_INSERT, "Hi"),
]

# Generate the patch string
dmp = diff_match_patch()
patches = dmp.patch_fromText(str(patch))
new_text, _ = dmp.patch_apply(patches, "Hello World!\n")
print(str(patch))
print(new_text)




@@ -1,5 +1,2 @@
-Hello
+Hi

Hi World!



In [44]:
from diff_match_patch import diff_match_patch, patch_obj
import urllib

In [50]:
with open("./app_setup.py", "r") as f:
    original_file = f.read()

# print(original_file[31:35])

patch = patch_obj()
patch.start1 = 31
patch.start2 = 31
patch.length1 = 43
patch.length2 = 41
patch.diffs = [
    (-1, 'print("Successfully connected to database")'),
    (1, 'print("Database connection established")'),
]
print(str(patch))
dmp = diff_match_patch()
new_patch = dmp.patch_fromText(str(patch))
new_text, results = dmp.patch_apply(new_patch, original_file)
print(results)
print(new_text)

@@ -32,43 +32,41 @@
-print(%22Successfully connected to database%22)
+print(%22Database connection established%22)

[False, False]
# app_setup.py
import os
import psycopg2
from psycopg2.extensions import connection
from agent.agent import CodingAgent
from agent.memory_manager import MemoryManager
from database.my_codebase import MyCodebase
from dotenv import load_dotenv
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from typing import Optional, List, Any, Callable
from pydantic import BaseModel
from agent.agent_functions import CommandPlan, FileChange

load_dotenv()
CODEAPP_DB_NAME = os.getenv("CODEAPP_DB_NAME")
CODEAPP_DB_USER = os.getenv("CODEAPP_DB_USER")
CODEAPP_DB_PW = os.getenv("CODEAPP_DB_PW")
CODEAPP_DB_HOST = os.getenv("CODEAPP_DB_HOST")
DIRECTORY = os.getenv("PROJECT_DIRECTORY")


def create_database_connection() -> connection:
    try:
        auth = {
            "dbname": CODEAPP_DB_NAME,
            "user": CODEAPP_DB_USER,
            "pas

In [34]:
with open("./app_setup.py", "r") as f:
    original_file = f.read()

# The new text after applying the changes
new_text = original_file.replace('print("Successfully connected to database")', 'print("Database connection established")')

# Initialize a diff_match_patch object
dmp = diff_match_patch()

# Compute the diffs between the original and new texts
diffs = dmp.diff_main(original_file, new_text)
print(diffs)
# Create a patch from the diffs
patches = dmp.patch_make(original_file, new_text, diffs)
print(patches[0].diffs)

# Apply the patch to the original text
patched_text, _ = dmp.patch_apply(patches, original_file)

print(patched_text)

[(0, '# app_setup.py\nimport os\nimport psycopg2\nfrom psycopg2.extensions import connection\nfrom agent.agent import CodingAgent\nfrom agent.memory_manager import MemoryManager\nfrom database.my_codebase import MyCodebase\nfrom dotenv import load_dotenv\nfrom fastapi import FastAPI\nfrom fastapi.middleware.cors import CORSMiddleware\nfrom typing import Optional, List, Any, Callable\nfrom pydantic import BaseModel\nfrom agent.agent_functions import CommandPlan, FileChange\n\nload_dotenv()\nCODEAPP_DB_NAME = os.getenv("CODEAPP_DB_NAME")\nCODEAPP_DB_USER = os.getenv("CODEAPP_DB_USER")\nCODEAPP_DB_PW = os.getenv("CODEAPP_DB_PW")\nCODEAPP_DB_HOST = os.getenv("CODEAPP_DB_HOST")\nDIRECTORY = os.getenv("PROJECT_DIRECTORY")\n\n\ndef create_database_connection() -> connection:\n    try:\n        auth = {\n            "dbname": CODEAPP_DB_NAME,\n            "user": CODEAPP_DB_USER,\n            "password": CODEAPP_DB_PW,\n            "host": CODEAPP_DB_HOST,\n        }\n        conn = psycopg2.c